import the libs.

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


web scraping useing requests lib and pandas.

In [4]:
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'

result=requests.get(url).content # call the url then get the content of it.

df=pd.read_html(result)[0] # get the first table

df = df.replace('Not assigned', np.nan).dropna() # remove the Not assigend cells/rows
df.tail(12) # show the last 12 row

,Postal Code,Borough,Neighbourhood
147,M4W,Downtown Toronto,Rosedale
148,M5W,Downtown Toronto,Stn A PO Boxes
151,M8W,Etobicoke,"Alderwood, Long Branch"
152,M9W,Etobicoke,"Northwest, West Humber - Clairville"
153,M1X,Scarborough,Upper Rouge
156,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
157,M5X,Downtown Toronto,"First Canadian Place, Underground city"
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


the shape of the data frame.

In [5]:
df.shape

(103, 3)

In [6]:
#read the "Geospatial_Coordinates.csv" then merage it with then old df. useing "Postal Code" as the join key.
df = pd.merge(pd.read_csv('Geospatial_Coordinates.csv'), df, on='Postal Code')
df

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [9]:
# Sorting
# set index for only Downtown Toronto
downtown_toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate post code column
downtown_toronto_data=downtown_toronto_data.drop(['Postal Code'], axis=1)
downtown_toronto_data.head()

,Latitude,Longitude,Borough,Neighbourhood
0,43.679563,-79.377529,Downtown Toronto,Rosedale
1,43.667967,-79.367675,Downtown Toronto,"St. James Town, Cabbagetown"
2,43.665860,-79.383160,Downtown Toronto,Church and Wellesley
3,43.654260,-79.360636,Downtown Toronto,"Regent Park, Harbourfront"
4,43.657162,-79.378937,Downtown Toronto,"Garden District, Ryerson"


In [10]:
# the shape of the data.
downtown_toronto_data.shape

(19, 4)

In [11]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'N5D1OHCDDUPOPMQDP3FO0G4ONA1C4D3AAP41EBFLNNBCVLDK' # your Foursquare ID
CLIENT_SECRET = 'J3NR3COA5A31333IRN5CCZL1GKEON2HJPIWRF4K0XI1R2VLC' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:N5D1OHCDDUPOPMQDP3FO0G4ONA1C4D3AAP41EBFLNNBCVLDK
CLIENT_SECRET:J3NR3COA5A31333IRN5CCZL1GKEON2HJPIWRF4K0XI1R2VLC


In [24]:
# fix columns names.
downtown_toronto_data.columns = ['Latitude', 'Longitude', 'Borough', 'Neighborhood']

In [25]:
neighborhood_latitude = downtown_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.3775294.


In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=N5D1OHCDDUPOPMQDP3FO0G4ONA1C4D3AAP41EBFLNNBCVLDK&client_secret=J3NR3COA5A31333IRN5CCZL1GKEON2HJPIWRF4K0XI1R2VLC&v=20180604&ll=43.6795626,-79.3775294&radius=500&limit=100'

In [27]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [28]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude']
                           , downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [29]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'],
                           downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

In [30]:
# A function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'])


Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [32]:
# the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [33]:
# check the venues that were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [34]:
# how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 126 uniques categories.


In [35]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [37]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0      Farmers Market  0.10
1            Beer Bar  0.10
2  Seafood Restaurant  0.10
3            Fountain  0.05
4         Cheese Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4     Sculpture Garden  0.06


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.25
1    Gastropub  0.05
2  Pizza Place  0.05
3   Poke Place  0.05
4         Park  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Candy Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
                venue  freq
0                 Pub  0.05
1         Pizza Place  0.05
2      Breakfast Spot  0.05
3     Bubble Tea Shop  0.05
4  Salon / Barbershop  0.05


----Comm

In [38]:
# pandas dataframe for the above results
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [39]:
# the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Beer Bar,Seafood Restaurant,Fountain,Cheese Shop,Basketball Stadium,French Restaurant,Bakery,Restaurant,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Sculpture Garden,Airport Food Court,Airport Gate,Coffee Shop,Harbor / Marina,Boat or Ferry
2,Central Bay Street,Coffee Shop,Gastropub,Pizza Place,Poke Place,Park,Café,Bubble Tea Shop,Sandwich Place,Modern European Restaurant,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Nightclub,Baby Store,Athletics & Sports,Coffee Shop,Restaurant,Italian Restaurant
4,Church and Wellesley,Pub,Pizza Place,Breakfast Spot,Bubble Tea Shop,Salon / Barbershop,Restaurant,Ramen Restaurant,Coffee Shop,Creperie,Beer Bar
5,"Commerce Court, Victoria Hotel",Café,Japanese Restaurant,Gastropub,Coffee Shop,Gym,Bakery,Gym / Fitness Center,Gluten-free Restaurant,Museum,Deli / Bodega
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym,Gastropub,Pub,Seafood Restaurant,Bookstore,Steakhouse,Bakery
7,"Garden District, Ryerson",Café,Plaza,Sandwich Place,Music Venue,Electronics Store,Burrito Place,Pizza Place,Sporting Goods Shop,Middle Eastern Restaurant,Mexican Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",Plaza,Park,Supermarket,Performing Arts Venue,Café,Salad Place,Deli / Bodega,Bubble Tea Shop,Dessert Shop,New American Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Wine Bar,Bakery,Coffee Shop,Cocktail Bar,Dessert Shop,Cheese Shop,Organic Grocery,Farmers Market


In [40]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 4, 1, 4, 4, 4, 1, 4], dtype=int32)

In [41]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head()

,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.679563,-79.377529,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Mexican Restaurant,Pizza Place,Pet Store,Performing Arts Venue,Organic Grocery,Opera House,Nightclub
1,43.667967,-79.367675,Downtown Toronto,"St. James Town, Cabbagetown",1,Café,Restaurant,Pub,Taiwanese Restaurant,Deli / Bodega,Butcher,Bakery,Caribbean Restaurant,Gastropub,Diner
2,43.665860,-79.383160,Downtown Toronto,Church and Wellesley,0,Pub,Pizza Place,Breakfast Spot,Bubble Tea Shop,Salon / Barbershop,Restaurant,Ramen Restaurant,Coffee Shop,Creperie,Beer Bar
3,43.654260,-79.360636,Downtown Toronto,"Regent Park, Harbourfront",4,Coffee Shop,Park,Breakfast Spot,Historic Site,Bakery,Chocolate Shop,Pub,Performing Arts Venue,Restaurant,Distribution Center
4,43.657162,-79.378937,Downtown Toronto,"Garden District, Ryerson",1,Café,Plaza,Sandwich Place,Music Venue,Electronics Store,Burrito Place,Pizza Place,Sporting Goods Shop,Middle Eastern Restaurant,Mexican Restaurant


In [42]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1 (Airport Lounge, Coffee Shop, Cafe, Restaurants & Grocery Store)

In [43]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,-79.383160,Pub,Pizza Place,Breakfast Spot,Bubble Tea Shop,Salon / Barbershop,Restaurant,Ramen Restaurant,Coffee Shop,Creperie,Beer Bar
10,-79.381576,Coffee Shop,Café,Japanese Restaurant,Gym,Bakery,Gym / Fitness Center,Deli / Bodega,Hotel,Restaurant,Sandwich Place
11,-79.379817,Café,Japanese Restaurant,Gastropub,Coffee Shop,Gym,Bakery,Gym / Fitness Center,Gluten-free Restaurant,Museum,Deli / Bodega


Cluster 2 (Gastropubs)

In [44]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,-79.367675,Café,Restaurant,Pub,Taiwanese Restaurant,Deli / Bodega,Butcher,Bakery,Caribbean Restaurant,Gastropub,Diner
4,-79.378937,Café,Plaza,Sandwich Place,Music Venue,Electronics Store,Burrito Place,Pizza Place,Sporting Goods Shop,Middle Eastern Restaurant,Mexican Restaurant
8,-79.384568,Coffee Shop,Seafood Restaurant,Hotel,Sushi Restaurant,Concert Hall,Plaza,Opera House,Café,Restaurant,Monument / Landmark


Cluster 3 (Cafes)

In [45]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,-79.400049,Café,Vietnamese Restaurant,Wine Bar,Bakery,Coffee Shop,Cocktail Bar,Dessert Shop,Cheese Shop,Organic Grocery,Farmers Market



Cluster 4 (Coffee Shop, Cafe, Park & Japanese Restaurant)¶


In [46]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-79.377529,Park,Playground,Trail,Mexican Restaurant,Pizza Place,Pet Store,Performing Arts Venue,Organic Grocery,Opera House,Nightclub
16,-79.382280,Café,Coffee Shop,Restaurant,Gym,Gastropub,Pub,Seafood Restaurant,Bookstore,Steakhouse,Bakery



Cluster 5 (Seafood, steakhouse, Hotel & Cafe)¶


In [47]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,-79.360636,Coffee Shop,Park,Breakfast Spot,Historic Site,Bakery,Chocolate Shop,Pub,Performing Arts Venue,Restaurant,Distribution Center
5,-79.375418,Gastropub,Coffee Shop,Café,Gym,BBQ Joint,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Food Truck,Creperie
6,-79.373306,Farmers Market,Beer Bar,Seafood Restaurant,Fountain,Cheese Shop,Basketball Stadium,French Restaurant,Bakery,Restaurant,Liquor Store
7,-79.387383,Coffee Shop,Gastropub,Pizza Place,Poke Place,Park,Café,Bubble Tea Shop,Sandwich Place,Modern European Restaurant,Middle Eastern Restaurant
9,-79.381752,Plaza,Park,Supermarket,Performing Arts Venue,Café,Salad Place,Deli / Bodega,Bubble Tea Shop,Dessert Shop,New American Restaurant
12,-79.400049,Bakery,Japanese Restaurant,Café,Bookstore,Yoga Studio,College Gym,Restaurant,Dessert Shop,Sandwich Place,Beer Bar
14,-79.394420,Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Sculpture Garden,Airport Food Court,Airport Gate,Coffee Shop,Harbor / Marina,Boat or Ferry
15,-79.374846,Beer Bar,Cocktail Bar,Seafood Restaurant,Farmers Market,French Restaurant,Park,Café,Restaurant,Museum,Food Truck
17,-79.422564,Grocery Store,Café,Park,Candy Store,Nightclub,Baby Store,Athletics & Sports,Coffee Shop,Restaurant,Italian Restaurant
18,-79.389494,Coffee Shop,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Portuguese Restaurant,Park,Creperie,Burrito Place,Diner,Discount Store
